# **Business Understanding**

### **Penjelasan Mengenai Dataset CDC Diabetes Health Indicators**



Pertama sedikit pengetahuan tentang diabetes. Diabetes adalah salah satu penyakit yang paling sering terjadi di seluruh dunia dan jumlah pasien diabetes terus bertambah selama bertahun-tahun. Penyebab utama diabetes masih belum diketahui, namun para ilmuwan percaya bahwa faktor genetik dan gaya hidup lingkungan berperan besar dalam terjadinya diabetes.

---
Dataset CDC Diabetes Health Indicators berisi mengenai statistik kesehatan dan informasi survei gaya hidup orang-orang secara umum beserta diagnosis diabetes mereka. 22 fitur terdiri dari beberapa demografi, hasil tes laboratorium, dan jawaban atas pertanyaan survei untuk setiap pasien. Variabel target untuk klasifikasi adalah apakah pasien menderita diabetes atau sehat yang berada pada fitur Diabetes_binary.

---


In [12]:
!pip install pycaret
!pip install dataprep

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

plt.style.use('seaborn-darkgrid')

from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score,precision_score, f1_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from dataprep.eda import plot_missing

<ipython-input-13-600ec31c78f3>:7: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-darkgrid')


In [14]:
import pandas as pd
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest

In [15]:
from google.colab import drive
drive.mount('/content/drive')

import os

# Pindah ke direktori yang sesuai di Google Drive
os.chdir('/content/drive/MyDrive/PSD/Diabetes Indicator/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Data Understanding**

# **Penjelasan Fitur**


**1.   Diabetes_binary**

    0 = no diabetes 1 =  diabetes(kategorikal)

**2.   HighBP**

    0 = no high BP 1 = high BP(kategorikal)

    High Blood Pressure(Tekanan Darah Tinggi) = Apakah tekanan darah pasien termasuk tinggi atau tidak?

    Tekanan darah adalah ukuran tekanan yang dihasilkan oleh aliran darah saat mengalir melalui pembuluh darah dalam tubuh.



**3. HighChol**

    0 = no high cholesterol 1 = high cholesterol(kategorikal)

    High Cholesterol = Apakah kolesterol pasien termasuk tinggi atau tidak

    Kolesterol adalah suatu jenis lemak yang ditemukan dalam tubuh manusia dan hewan.
    Ini adalah komponen penting dari membran sel dan berperan dalam berbagai fungsi biologis.
    Kolesterol adalah senyawa lipid yang memiliki struktur kimia khusus yang dikenal sebagai sterol.

**4. CholCheck**

    0 = no cholesterol check in 5 years 1 = yes cholesterol check in 5 years(kategorikal)

    Cholesterol Check = Apakah kolesterol di cek dalam 5 tahun terakhir atau tidak

**5. BMI**

    Body Mass Index = Berat badan pasien(Numerik)

**6. Smoker**

    Apakah Anda merokok setidaknya 100 batang seumur hidup Anda?
    0 = no 1 = yes(kategorikal)

**7. Stroke**

    (Pernah diberitahu) Anda terkena stroke?.
    0 = no 1 = yes(kategorikal)

**8. HeartDiseaseorAttack**

    coronary heart disease (CHD) or myocardial infarction (MI)
    Apakah anda memiliki penyakit jantung?
    0 = no 1 = yes(kategorikal)
    ---
    Coronary Artery Disease (CAD)(yaitu, jantung koroner) merupakan suatu gangguan fungsi jantung
    yang disebabkan karena otot miokard kekurangan suplai darah
    akibat adanya penyempitan arteri koroner dan tersumbatnya pembuluh darah jantung.
    Jika Anda tidak tahu apakah memiliki penyakit jantung koroner,
    maka Anda dapat melakukan tes kesehatan untuk mengetahui apakah Anda menderita penyakit jantung koroner.
    ---
    myocardial infarction (MI)(yaitu, serangan jantung) adalah
    kematian permanen (nekrosis) otot jantung akibat kekurangan pasokan oksigen (iskemia) yang berkepanjangan.
    Jika Anda tidak tahu apakah memiliki penyakit serangan jantung,
    maka Anda dapat melakukan tes kesehatan untuk mengetahui apakah Anda menderita penyakit serangan jantung.

**9. PhysActivity**

    Apakah anda memiliki aktivitas fisik dalam 30 hari terakhir?(tidak termasuk kerja)
    0 = no 1 = yes(kategorikal)

**10. Fruits**

    Apakah Anda Mengonsumsi Buah 1 kali atau lebih dalam sehari?
    0 = no 1 = yes(kategorikal)

**11. Veggies**

    Apakah Anda Mengkonsumsi Sayuran 1 kali atau lebih per hari?
    0 = no 1 = yes(kategorikal)

**12. HvyAlcoholConsump**

    Apakah anda merupakan pemabuk berat
    (jika pria dewasa lebih dari 14 gelas per minggu dan wanita dewasa lebih dari 7 gelas per minggu) ?
    0 = no 1 = yes(kategorikal)

**13. AnyHealthcare**

    Memiliki jaminan perawatan kesehatan apa pun, termasuk asuransi kesehatan, paket prabayar seperti HMO, dll?
    0 = no 1 = yes(kategorikal)

**14. NoDocbcCost**

    Apakah dalam 12 bulan terakhir ada saat di mana Anda perlu ke dokter tetapi tidak bisa karena biaya?
    0 = no 1 = yes(kategorikal)

**15. GenHlth**

    Menurut Anda, secara umum kesehatan Anda adalah: dalam skala 1-5
    1 = excellent 2 = very good 3 = good 4 = fair 5 = poor(kategorikal)

**16. MentHlth**

    Sekarang coba pikirkan tentang kesehatan mental Anda, yang meliputi
    stres, depresi, dan masalah dengan emosi,
    selama berapa hari dalam 30 hari terakhir kesehatan mental Anda tidak baik?
    dalam skala 1-30 hari

**17. PhysHlth**

    Sekarang pikirkan tentang kesehatan fisik Anda, yang mencakup penyakit fisik dan cedera,
    selama berapa hari dalam 30 hari terakhir kesehatan fisik Anda tidak baik?
    dalam skala 1-30 hari

**18. DiffWalk**

    Apakah Anda mengalami kesulitan saat berjalan atau menaiki tangga?
    0 = no 1 = yes(kategorikal)

**19. Sex**

    0 = female 1 = male(kategorikal)

**20. Age**

    13-level Umur category(Kategorikal)
    1 = Umur 18 sampai 24
    2 = Umur 25 sampai 29
    3 = Umur 30 sampai 34
    4 = Umur 35 sampai 39
    5 = Umur 40 sampai 44
    6 = Umur 45 sampai 49
    7 = Umur 50 sampai 54
    8 = Umur 55 sampai 59
    9 = Umur 60 sampai 64
    10 = Umur 65 sampai 69
    11 = Umur 70 sampai 74
    12 = Umur 75 sampai 79
    13 = Umur 80 atau lebih tua

**21. Education**

    Tingkat pendidikan skala 1-6 (kategorikal)
    1 = Tidak pernah bersekolah atau hanya taman kanak-kanak
    2 = Kelas 1 sampai 8 (SD)
    3 = Kelas 9 hingga 11 (Beberapa sekolah menengah)
    4 = Kelas 12 atau GED (Lulusan sekolah menengah atas)
    5 = Kuliah 1 tahun hingga 3 tahun (Beberapa perguruan tinggi atau sekolah teknik)
    6 = Perguruan tinggi 4 tahun atau lebih (Lulusan perguruan tinggi)

**22. Income**

    Skala pendapatan skala 1-8 (kategorikal)
    1 Kurang dari $10.000 (kurang dari Rp. 10.000.000,-)
    2 Kurang dari $15.000 ($10.000 hingga kurang dari $15.000)
    3 Kurang dari $20.000 ($15.000 hingga kurang dari $20.000)
    4 Kurang dari $25.000 ($20.000 hingga kurang dari $25.000)
    5 Kurang dari $35.000 ($25.000 hingga kurang dari $35.000)
    6 Kurang dari $50.000 ($35.000 hingga kurang dari $50.000)
    7 Kurang dari $75.000 ($50.000 hingga kurang dari $75.000)
    8 $75.000 atau lebih

In [16]:
diabet = pd.read_csv('/content/drive/MyDrive/PSD/Diabetes Indicator/diabetes_binary_health_indicators_BRFSS2015.csv')
diabet

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253675,0.0,1.0,1.0,1.0,45.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,3.0,0.0,5.0,0.0,1.0,5.0,6.0,7.0
253676,1.0,1.0,1.0,1.0,18.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,4.0,0.0,0.0,1.0,0.0,11.0,2.0,4.0
253677,0.0,0.0,0.0,1.0,28.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,5.0,2.0
253678,0.0,1.0,0.0,1.0,23.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,7.0,5.0,1.0


**Pengecekan Missing Value**

Pengecekan missing value pada dataset, pengecekan dilakukan karena hal ini akan berpengaruh terhadap analisa dan performansi model machine learning jika data-data tersebut dijadikan “modal” dalam membuat suatu model Machine Learning.

In [17]:
#  data understanding
plot_missing(diabet)

Output hidden; open in https://colab.research.google.com to view.

**Melihat Ada Berapa Kelas Dalam Data**

In [18]:
IsDiabet = diabet['Diabetes_binary'].unique()
print('Jumlah Kelas :',IsDiabet)

Jumlah Kelas : [0. 1.]


**Pengecekan Data Setiap Kelas**

Mengecek data pada setiap kelas yaitu data pasien yang terkena diabetes dan data pasien yang tidak terkena diabetes. Untuk mengetahui apakah data antara 2 kelas tersebut seimbang atau tidak, karena data yang tidak seimbang dapat mempengaruhi akurasi dari klasifikasi.

In [19]:
jumlah_diabet = diabet['Diabetes_binary'].value_counts().get(1, 0)
jumlah_nondiabet = diabet['Diabetes_binary'].value_counts().get(0, 0)
print('Jumlah pasien diabet : ',jumlah_diabet)
print('Jumlah pasien non-diabet : ',jumlah_nondiabet)

Jumlah pasien diabet :  35346
Jumlah pasien non-diabet :  218334


# **Pre-Processing**

**SMOTE (Synthetic Minority Over-sampling Technique) :**

Kegunaan utama dari library atau algoritma SMOTE adalah untuk mengatasi masalah ketidakseimbangan kelas dalam dataset. Masalah ketidakseimbangan kelas terjadi ketika ada perbedaan yang signifikan dalam jumlah sampel antara kelas mayoritas dan kelas minoritas dalam dataset. seperti jika data pasien diabetes lebih banyak dari non-diabetes.**Metode SMOTE bekerja dengan mencari k nearest neighbors untuk setiap data di kelas minoritas, setelah itu dibuat data sintetis sebanyak persentase duplikasi yang diinginkan antara data minor dan knearest neighbors yang dipilih secara acak**. berikut merupakan rumus manual dari SMOTE (Synthetic Minority Over-sampling Technique :

$\text{SMOTE} : (X_{new} - X_{min}) = \lambda \cdot (X - X_{min}) $

Di mana:
- $ X_{new} $ adalah sampel sintetis yang dihasilkan.
- $ X_{min} $ adalah sampel dari kelas minoritas yang sedang diproses.
- $ X $ adalah salah satu dari k-nearest neighbors dari $ X_{min} $ yang berbeda dari $ X_{min} $.
- $ \lambda $ adalah suatu bilangan acak antara 0 dan 1.



In [20]:
X = diabet.drop(['Diabetes_binary'],axis=1)
y = diabet['Diabetes_binary']
feature_names = diabet.columns
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)
feature_names

Index(['Diabetes_binary', 'HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker',
       'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education',
       'Income'],
      dtype='object')

**Information Gain :**

Information Gain adalah salah satu metode penyeleksian fitur, yang menyeleksi dengan cara mengukur peningkatan dalam ketidakpastian (entropi) dari dataset yang disebabkan oleh pengetahuan tentang suatu fitur. Semakin besar peningkatannya, semakin informatif fitur tersebut terhadap pemisahan atau klasifikasi data. berikut merupakan rumus manual dari Information Gain :

$
 \text{Information Gain} = H(D) - H(D|A)
$

Di mana :
- $ H(D) $ adalah entropi dataset sebelum informasi tambahan dari fitur $ A $ digunakan
- $ H(D|A) $ adalah entropi dataset setelah fitur $ A $ digunakan untuk pemisahan atau klasifikasi.


In [21]:
# Seleksi fitur menggunakan Information Gain
selector = SelectKBest(score_func=mutual_info_classif, k='all') #k = all berfungsi untuk menghitung semua fitur dengan information gain
X_new = selector.fit_transform(X, y)

# Mengambil indeks fitur yang terpilih
selected_features = selector.get_support(indices=True)

# Hasil seleksi fitur beserta nama fitur dan pentingnya
feature_names = [feature_names[i] for i in selected_features]
feature_scores = selector.scores_

# Membuat DataFrame untuk menampung nama fitur dan nilainya dalam satu frame
feature_info_gain_df = pd.DataFrame({'Fitur': feature_names, 'Information Gain': feature_scores})

# Menampilkan fitur-fitur secara berurutan berdasarkan Information Gain yang terbesar
sorted_features = feature_info_gain_df.sort_values(by='Information Gain', ascending=False)
print(sorted_features)

                   Fitur  Information Gain
13           NoDocbcCost          0.214620
18                   Sex          0.210766
3              CholCheck          0.210239
0        Diabetes_binary          0.200825
1                 HighBP          0.167216
20             Education          0.165133
19                   Age          0.151587
7   HeartDiseaseorAttack          0.128163
8           PhysActivity          0.124682
4                    BMI          0.120320
9                 Fruits          0.112200
17              DiffWalk          0.109034
16              PhysHlth          0.096620
15              MentHlth          0.087826
11     HvyAlcoholConsump          0.073129
6                 Stroke          0.071450
14               GenHlth          0.056744
2               HighChol          0.056621
12         AnyHealthcare          0.041062
5                 Smoker          0.034518
10               Veggies          0.022357


**Standard Scaler**

Standart Scaler adalah bagian dari scikit-learn (sklearn) dan digunakan untuk melakukan normalisasi fitur dalam data Anda. Standart Scaler mengubah distribusi nilai fitur sehingga memiliki rata-rata (mean) nol dan deviasi standar (standard deviation) satu. Ini berarti data fitur akan memiliki nilai tengah yang sama (nol) dan tersebar dengan baik dengan deviasi standar yang seragam, yang membantu algoritma machine learning yang sensitif terhadap skala berkinerja lebih baik. berikut merupakan rumus manual dari standard scaler :

$$
\text{StandardScaler: } Z = \frac{X - \mu}{\sigma}
$$


In [22]:
X = diabet.drop(['Diabetes_binary','AnyHealthcare','Smoker', 'Veggies'],axis=1)
y = diabet['Diabetes_binary']
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

# Splitting data into training and test set:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creating StandardScaler instance
sc = StandardScaler()
# Fitting Standard Scaller
X_train = sc.fit(X_train)
with open('StandardScaler.pkl', 'wb') as file:
    pickle.dump(X_train, file)
# Scaling data
X_test = sc.transform(X_test)


# **Modelling**

# **PyCaret**

Library PyCaret adalah kumpulan library open source yang digunakan untuk membangun model machine learning dengan mudah. PyCaret menyediakan tools untuk preprocessing data, membangun model, membandingkan model, mengevaluasi, menampilkan visualisasi hasil training sampai menyimpan model dalam
format tertentu. PyCaret memungkinkan untuk menguji banyak model secara mudah dan cepat. Maka dari itu dalam proses untuk mendapatkan model terbaik, penelitian ini menggunakan library PyCaret.

In [23]:
from pycaret.datasets import get_data
from pycaret.classification import *

Melakukan setup data untuk proses modeling dengan menggunakan parameter diabet sebagai dataset yang akan digunakan dan target.

In [24]:
s = setup(diabet, target='Diabetes_binary')

,Description,Value
0,Session id,1799
1,Target,Diabetes_binary
2,Target type,Binary
3,Original data shape,"(253680, 22)"
4,Transformed data shape,"(253680, 22)"
5,Transformed train set shape,"(177576, 22)"
6,Transformed test set shape,"(76104, 22)"
7,Numeric features,21
8,Preprocess,True
9,Imputation type,simple


**Compare Models**

Pada library PyCaret terdapat sekitar 16 algoritma klasifikasi yang tersedia untuk proses perbandingan metode. Algoritma klasifikasi tersebut antara lain Logistic Regression, K Neighbors Classifier, Decision Tree Classifier, Quadratic
Discriminant Analysis, Linear Discriminant Analysis, Light Gradient Boosting Machine, Gradient Boosting Classifier, Naive Bayes, Extra Trees Classifier, CatBoost Classifier, Extreme Gradient Boosting, Random Forest Classifier,
AdaBoost Classifier, Ridge Classifier, SVM - Linear Kernel, dan Dummy Classifier. Dari beberapa metode tersebut, akan dilakukan perbandingan atau komparasi dengan menggunakan fungsi compare_models(). Fungsi ini berguna untuk melatih dan mengevaluasi performa model algoritma klasifikasi dengan menggunakan cross validation. Keluaran dari fungsi ini adalah rata-rata skor dari cross validation.

In [ ]:
best = s.compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.8668,0.8301,0.1633,0.5788,0.2546,0.2059,0.2566,8.9490
gbc,Gradient Boosting Classifier,0.8664,0.8301,0.1672,0.5711,0.2586,0.2086,0.2570,21.3390
ada,Ada Boost Classifier,0.8649,0.8267,0.1915,0.5435,0.2832,0.2270,0.2651,4.8020
xgboost,Extreme Gradient Boosting,0.8645,0.8254,0.1706,0.5437,0.2597,0.2069,0.2496,2.1220
lr,Logistic Regression,0.8632,0.8223,0.1552,0.5307,0.2401,0.1890,0.2329,7.7110
ridge,Ridge Classifier,0.8625,0.0000,0.0395,0.6042,0.0742,0.0580,0.1287,0.2850
lda,Linear Discriminant Analysis,0.8614,0.8196,0.2059,0.5069,0.2928,0.2309,0.2599,1.4820
dummy,Dummy Classifier,0.8607,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.1260
rf,Random Forest Classifier,0.8598,0.7985,0.1704,0.4915,0.2530,0.1953,0.2291,20.7470
et,Extra Trees Classifier,0.8515,0.7767,0.1742,0.4205,0.2463,0.1793,0.2009,27.1720


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

In [ ]:
s.predict_model(best)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.8665,0.8292,0.1610,0.5749,0.2515,0.2029,0.2534


,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,Diabetes_binary,prediction_label,prediction_score
110959,0.0,0.0,1.0,18.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,4.0,6.0,8.0,0.0,0,0.9978
148493,0.0,1.0,1.0,29.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,7.0,1.0,0.0,8.0,6.0,8.0,0.0,0,0.9376
77022,0.0,0.0,1.0,19.0,1.0,0.0,0.0,1.0,0.0,1.0,...,15.0,0.0,0.0,0.0,4.0,4.0,3.0,0.0,0,0.9767
192864,1.0,1.0,1.0,26.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,10.0,5.0,6.0,0.0,0,0.8837
187801,0.0,0.0,0.0,28.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,9.0,6.0,7.0,0.0,0,0.9924
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199797,0.0,0.0,1.0,29.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,6.0,5.0,8.0,0.0,0,0.9924
179610,0.0,1.0,1.0,40.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,7.0,1.0,0.0,9.0,6.0,8.0,0.0,0,0.9534
169118,0.0,0.0,1.0,28.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,8.0,4.0,8.0,0.0,0,0.9259
57464,0.0,0.0,1.0,36.0,0.0,1.0,0.0,1.0,1.0,1.0,...,10.0,15.0,0.0,1.0,13.0,4.0,4.0,0.0,0,0.5895


## Light Gradient Boosting Machine (LightGBM)

Light Gradient Boosting Machine (LightGBM) adalah framework gradient-boosting yang didasarkan pada pohon keputusan untuk meningkatkan efisiensi model dan mengurangi penggunaan memori.

### Cara Kerja LightGBM

- **Gradient Boosting**: LightGBM adalah algoritma ensemble berbasis gradient boosting. Algoritma ini bekerja dengan menggabungkan beberapa pohon keputusan (biasanya pohon yang dangkal) untuk meningkatkan performa prediksi.

- **Leaf-Wise Splitting**: LightGBM menggunakan leaf-wise splitting, yang memungkinkan untuk membangun pohon dengan sedikit node yang lebih dalam. Ini berbeda dari level-wise splitting yang digunakan oleh banyak algoritma pohon keputusan lainnya.

- **Histogram-based Learning**: LightGBM menggunakan histogram-based learning, yaitu mengelompokkan data menjadi interval nilai (histogram) untuk mengurangi kompleksitas perhitungan dan meningkatkan efisiensi.

- **Parameter Kontrol Overfitting**: Anda dapat mengendalikan tingkat overfitting dengan mengatur parameter seperti `max_depth`, `min_child_samples`, dan lain-lain.

- **Dukungan GPU**: LightGBM mendukung pelatihan pada GPU, yang dapat mempercepat proses pelatihan.

### Arsitektur LightBGM
LightBGM membagi tree leaf-wise yang berlawanan dengan algoritma boosting lainnya yang menumbuhkan tree level-wise. Algoritma ini memilih daun dengan delta loss maksimum untuk tumbuh. Karena daunnya tetap, algoritma leaf-wise memiliki kerugian yang lebih rendah dibandingkan dengan algoritma level-wise. Pertumbuhan pohon secara leaf-wise dapat meningkatkan kompleksitas model dan dapat menyebabkan overfitting pada dataset yang kecil.
Di bawah ini adalah representasi diagram dari Pertumbuhan Pohon Bijaksana Daun:

![LGBM](https://media.geeksforgeeks.org/wp-content/uploads/20200520035248/Leaf-Wise-Tree-Growth.png)



### Rumus

Secara umum, LightGBM tidak memiliki rumus matematis seperti yang dimiliki oleh beberapa algoritma lain seperti regresi linier. Namun, salah satu rumus utama yang digunakan dalam algoritma ensemble ini adalah rumus untuk perhitungan nilai prediksi dari sejumlah pohon keputusan:


$ \hat{y} = \sum_{i=1}^{n} f_i(x) $

Di mana:
- $\hat{y}$ adalah nilai prediksi.
- $n$ adalah jumlah pohon dalam ensemble.
- $f_i(x)$ adalah fungsi prediksi dari pohon keputusan ke-$i$ yang diaplikasikan pada data $x$.

### Penggunaan

LightGBM sangat cocok untuk dataset besar dan tugas klasifikasi atau regresi yang kompleks. Ini adalah salah satu algoritma yang populer dalam kompetisi data science dan digunakan dalam berbagai aplikasi machine learning.

In [ ]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score

# **Parameter LightGBM**

parameter pada inisialisasi metoode LightGBM adalah parameter yang digunakan untuk mengonfigurasi dan mengendalikan perilaku model LightGBM. Parameters ini membantu Anda menyesuaikan model LightGBM sesuai dengan kebutuhan dan karakteristik data Anda.

Parameter-parameter ini dapat mencakup berbagai aspek seperti:

- **Parameter Model**: Parameter yang mempengaruhi struktur dan karakteristik model, seperti **objective** (tujuan tugas, misalnya "binary" untuk klasifikasi biner), **num_leaves** (jumlah daun dalam setiap pohon), **learning_rate** (tingkat pembelajaran), **n_estimators** (jumlah pohon dalam ensemble), dan lain-lain.

- **Parameter Pelatihan**: Parameter yang memengaruhi proses pelatihan model, seperti **max_depth** (kedalaman maksimum setiap pohon), **min_child_samples** (jumlah sampel minimum di setiap leaf node), **early_stopping_rounds** (berhenti dini jika performa tidak membaik dalam sejumlah iterasi tertentu), dan lain-lain.

- **Parameter Penyusunan** Model: Parameter yang memengaruhi bagaimana model dianalogikan, seperti **metric** (metrik evaluasi yang digunakan selama pelatihan), **categorical_feature** (fitur-fitur kategori yang harus diperlakukan secara khusus), dan lain-lain.

- **Parameter Komputasi**: Parameter yang memengaruhi pengaturan komputasi, seperti penggunaan GPU (**device**), **nthread** (jumlah thread yang digunakan selama pelatihan), dan lain-lain.

#**Parameter terbaik yang akan dipakai menurut rekomendasi dari PyCaret**

- boosting_type='gbdt': Jenis boosting yang digunakan. 'gbdt' adalah Gradient Boosting Decision Tree, yang merupakan jenis boosting paling umum.

- class_weight=None: Pilihan untuk menangani ketidakseimbangan kelas. Jika Anda memiliki masalah klasifikasi yang tidak seimbang, Anda dapat mengaturnya menjadi 'balanced' atau menyediakan bobot kelas secara manual.

- colsample_bytree=1.0: Persentase fitur yang akan digunakan pada setiap iterasi. Nilai 1.0 berarti menggunakan semua fitur.

- importance_type='split': Jenis penghitungan pentingnya fitur. 'split' menghitung berapa kali suatu fitur digunakan dalam pemisahan simpul, sedangkan 'gain' menghitung gain informasi.

- learning_rate=0.1: Tingkat pembelajaran atau laju belajar model. Ini menentukan seberapa besar kontribusi dari setiap pohon dalam ensemble.

- max_depth=-1: Kedalaman maksimum dari setiap pohon. -1 berarti tidak ada batasan pada kedalaman pohon.

- min_child_samples=20: Jumlah sampel minimum yang diperlukan untuk membentuk simpul daun dalam pohon.

- min_child_weight=0.001: Berat minimum yang diperlukan pada setiap anak simpul.

- min_split_gain=0.0: Gain minimum yang diperlukan untuk membagi simpul.

- n_estimators=100: Jumlah pohon atau iterasi yang akan dilatih.

- n_jobs=-1: Jumlah pekerja atau inti CPU yang akan digunakan dalam pelatihan. -1 berarti menggunakan semua inti yang tersedia.

- num_leaves=31: Jumlah daun atau simpul terminal pada setiap pohon.

- objective=None: Tujuan dari model. Jika tidak ditentukan, maka itu akan diambil dari parameter boosting_type.

- random_state=4329: Seed untuk menghasilkan angka acak untuk reproduktibilitas.

- reg_alpha=0.0 dan reg_lambda=0.0: Parameter regularisasi. Menambahkan regularisasi membantu mencegah overfitting.

- subsample=1.0: Persentase sampel yang akan digunakan pada setiap iterasi. Nilai 1.0 berarti menggunakan semua sampel.

- subsample_for_bin=200000: Jumlah sampel yang digunakan untuk membangun histogram pada data kategorikal.

- subsample_freq=0: Frekuensi pengecekan pengambilan sampel. Jika non-nol, itu menjadi jumlah langkah antara pengambilan sampel.

In [ ]:
# Inisialisasi model LightGBM dengan params yang direkomendasikan oleh pycaret
model_lgbm = lgb.LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,importance_type='split', learning_rate=0.1, max_depth=-1,min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,random_state=4329, reg_alpha=0.0, reg_lambda=0.0, subsample=1.0,subsample_for_bin=200000, subsample_freq=0)

In [ ]:
# Melatih model pada data pelatihan
model_lgbm.fit(X_train, y_train)
with open('LightGBM.pkl', 'wb') as file:
    pickle.dump(model_lgbm, file)

# Melakukan prediksi pada data pengujian
y_pred = model_lgbm.predict(X_test)

ValueError: ignored

In [ ]:
# Mengukur performa model
accuracy = accuracy_score(y_test, y_pred)
print("Akurasi model: {:.2f}%".format(accuracy * 100))

In [ ]:
display(y_train,X_train)

# **Testing Model**

In [ ]:
data_ke = X.iloc[57257]
print(data_ke)

In [ ]:
data_lgbm =[[0.0,0.0,1.0,29.0,0.0,0.0,1.0,1.0,0.0,0.0,5.0,30.0,30.0,0.0,0.0,5.0,6.0,8.0]] # target seharusnya = 0

y_prediksi = model_lgbm.predict(data_lgbm)
y_prediksi

In [ ]:
data_lgbm =[[0.0,0.0,1.0,22.0,0.0,0.0,1.0,1.0,0.0,1.0,4.0,1.0,1.0,0.0,1.0,7.0,5.0,5.0]] # target seharusnya = 0

y_prediksi = model_lgbm.predict(data_lgbm)
y_prediksi

In [ ]:
data_lgbm =[[1.0,1.0,1.0,30.0,0.0,1.0,0.0,1.0,0.0,0.0,5.0,30.0,30.0,1.0,0.0,9.0,5.0,1.0]] # target seharusnya = 1

y_prediksi = model_lgbm.predict(data_lgbm)
y_prediksi

In [ ]:
data_lgbm =[[0.0,0.0,0.0,25.0,0.0,0.0,1.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0]] # target seharusnya = 0

y_prediksi = model_lgbm.predict(data_lgbm)
y_prediksi

In [ ]:
!jupyter nbconvert --execute --to html "/content/PSD_Proyek2.ipynb"